In [2]:
import femm
import numpy as np
import cmath
import scipy.optimize as opt
import matplotlib.pyplot as plt

# The package must be initialized with the openfemm command.
femm.openfemm()

# We need to create a new Magnetostatics document to work on.
femm.newdocument(0)

In [20]:
Description_Txt   = "TypeA_VC"
outputfile        = 'TypeA_VC.out'
Parameters = {
    'InnCoil_Circuit': 'InnCoil_Circuit',
    'OutCoil_Circuit': 'OutCoil_Circuit',
    'InnCoil_Curr': 0.00, # Normal LVDT: put current on inner coil
    'OutCoil_Curr': 0.02, # Reversed LVDT: put current on outer coils
    'Sim_Freq': 0, # AC frequency = 0: simulate DC situation
    'NSteps': 10,
    'StepSize': 1,
    'InnCoil_Offset': -5
}
# 32 AWG
# 0.249 = 0.2032 + 0.0229*2
InnCoil_para = {
    'WireType': "32 AWG",
    'WireDiam': 0.2032,
    'WireInsul': 0.0229,
    'Layers': 6,
    'InnRadius': 11,
    'Height': 24,
    'InnCoil_Dist': 0, # offset distance for inner coil
}

OutCoil_para = {
    'WireType': "32 AWG",
    'WireDiam': 0.2032,
    'WireInsul': 0.0384,
    'Layers': 7,
    'InnRadius': 35,
    'Height': 13.5,
    'OutCoil_Dist': 54.5,# distance between mid points of outer coils. Space between coils is then OutCoil_Distance - OutCoil_Height
}

Magnet_para = {
    'Magnet_Length': 40,
    'Magnet_Diameter': 10,
    'Magnet_Material': "N40",
    'Magnet_VertShift': 0
}
# Define the problem type.  
# Magnetostatic; 
# Units of mm; 
# Axisymmetric; 
# Precision of 10^(-10) for the linear solver
femm.mi_probdef( Parameters['Sim_Freq'], 'millimeters', 'axi', 1.0e-10)

In [19]:
InnCoil_para['Diam_withInsul'] = round(InnCoil_para['WireDiam'] + 2*InnCoil_para['WireInsul'],3)
InnCoil_para['OutRadius'] = InnCoil_para['InnRadius'] + (InnCoil_para['NrWireLayers']*(InnCoil_para['WireDiam'] + 2*InnCoil_para['WireInsul']))
InnCoil_para['HighEnd'] = (InnCoil_para['InnCoil_Dist'] + InnCoil_para['Height'])/2
InnCoil_para['LowEnd'] = (InnCoil_para['InnCoil_Dist'] - InnCoil_para['Height'])/2
InnCoil_para['windingNum_single'] = InnCoil_para['Height'] / InnCoil_para['InnCoil_Diam_withInsul']
InnCoil_para['WindingNum_total']  = InnCoil_para['NrWireLayers'] * InnCoil_para['InnCoil_windingNum_single']

print("InnCoil_Diam_withInsul: ", InnCoil_para['InnCoil_Diam_withInsul'],
        "\nInnCoil_InnRadius: ", InnCoil_para['InnCoil_InnRadius'],
        "\nInnCoil_OutRadius: ", InnCoil_para['InnCoil_OutRadius'],
        "\nInnCoil_LowEnd: ", InnCoil_para['InnCoil_LowEnd'],
        "\nInnCoil_HighEnd: ", InnCoil_para['InnCoil_HighEnd'],
        "\nInnCoil_num_Winding_1layer: ", InnCoil_para['InnCoil_windingNum_single'],
        "\nInnCoil_num_Winding_total: ", InnCoil_para['InnCoil_WindingNum_total'],
        "\n")

OutCoil_para['OuterCoil_Diam_withInsul'] = round(OutCoil_para['WireDiam'] + 2*OutCoil_para['WireInsul'],3)
OuterCoil_Diam_withInsul = round(OutCoil_para['WireDiam'] + 2*OutCoil_para['WireInsul'],3)
OuterCoil_OutRadius = OutCoil_para['InnRadius'] + (OutCoil_para['NrWireLayers']*(OutCoil_para['WireDiam'] + 2*OutCoil_para['WireInsul']))
OuterCoil_LowEnd = (OutCoil_para['OutCoil_Dist'] - OutCoil_para['Height'])/2
OuterCoil_HighEnd = (OutCoil_para['OutCoil_Dist'] + OutCoil_para['Height'])/2
OuterCoil_single_layer_winding = OutCoil_para['Height'] / OuterCoil_Diam_withInsul
OuterCoil_winding  = OutCoil_para['NrWireLayers'] * OuterCoil_single_layer_winding

UppOutCoil_highEnd = OuterCoil_HighEnd
UppOutCoil_lowEnd = OuterCoil_LowEnd
LowOutCoil_highEnd = -OuterCoil_LowEnd
LowOutCoil_lowEnd = -OuterCoil_HighEnd
UppOutCoil_Circuit = "UppOutCoil_Circuit"
LowOutCoil_Circuit = "LowOutCoil_Circuit"

print("OuterCoil_Diam_withInsul: ", OuterCoil_Diam_withInsul,
        "\nOuterCoil_InRadius: ", OuterCoil_InnRadius,
        "\nOuterCoil_OutRadius: ", OuterCoil_OutRadius,
        "\nOuterCoil_LowEnd: ", OuterCoil_LowEnd,
        "\nOuterCoil_HighEnd: ", OuterCoil_HighEnd,
        "\nOuterCoil_num_Winding_1layer: ", OuterCoil_windingNum_single,
        "\nOuterCoil_num_Winding_total: ", OuterCoil_WindingNum_total,
        "\nUppOutCoil_highEnd: ", UppOutCoil_highEnd,
        "\nUppOutCoil_lowEnd: ", UppOutCoil_lowEnd,
        "\nLowOutCoil_highEnd: ", LowOutCoil_highEnd,
        "\nLowOutCoil_lowEnd: ", LowOutCoil_lowEnd,
        "\n")

InnCoil_wirelength = 0 
for i in range(InnCoil_para['NrWireLayers']):
    InnCoil_wirelength += 2*cmath.pi*(InnCoil_InnRadius + i*InnCoil_Diam_withInsul)*InnCoil_windingNum_single
print("InnCoil_wirelength: ", InnCoil_wirelength,"\n")
OuterCoil_wirelength = 0
for i in range(OutCoil_para['NrWireLayers']):
    OuterCoil_wirelength += 2*cmath.pi*(OuterCoil_InnRadius + i*OuterCoil_Diam_withInsul)*OuterCoil_windingNum_single
print("OuterCoil_wirelength: ", OuterCoil_wirelength,"\n")

Magnet_UppEnd = Magnet_para['Magnet_Length'] / 2 + Magnet_para['Magnet_VertShift']
Magnet_LowEnd = -Magnet_para['Magnet_Length'] / 2 + Magnet_para['Magnet_VertShift']
Magnet_Radius = Magnet_para['Magnet_Diameter'] / 2
print("Magnet_UppEnd: ", Magnet_UppEnd,
        "\nMagnet_LowEnd: ", Magnet_LowEnd,
        "\nMagnet_Radius: ", Magnet_Radius)

InnCoil_Diam_withInsul:  0.249 
InnCoil_InRadius:  11 
InnCoil_OutRadius:  12.494 
InnCoil_HighEnd:  12.0 
InnCoil_LowEnd:  -12.0 
InnCoil_num_Winding_1layer:  96.3855421686747 
InnCoil_num_Winding_total:  578.3132530120482 

OuterCoil_Diam_withInsul:  0.28 
OuterCoil_InRadius:  35 
OuterCoil_OutRadius:  36.96 
OuterCoil_LowEnd:  20.5 
OuterCoil_HighEnd:  34.0 
OuterCoil_num_Winding_1layer:  48.21428571428571 
OuterCoil_num_Winding_total:  337.49999999999994 
UppOutCoil_highEnd:  34.0 
UppOutCoil_lowEnd:  20.5 
LowOutCoil_highEnd:  -20.5 
LowOutCoil_lowEnd:  -34.0 

InnCoil_wirelength:  42232.0893875825 

OuterCoil_wirelength:  76001.40947564426 

Magnet_UppEnd:  20.0 
Magnet_LowEnd:  -20.0 
Magnet_Radius:  5.0


In [ ]:
femm.mi_drawrectangle(InnCoil_para['InRadius'], InnCoil_HighEnd, InnCoil_OutRadius, InnCoil_LowEnd)
femm.mi_addcircprop(Parameters['InnCoil_Circuit'], Parameters['InnCoil_Curr'], 1)


femm.mi_clearselected()
femm.mi_selectrectangle(InnCoil_InRadius,InnCoil_UppEnd, InnCoil_OutRadius, InnCoil_LowEnd, 4)
femm.mi_setgroup(1)
femm.mi_clearselected()
femm.mi_addblocklabel(InnCoil_InnRadius + InnCoil_WireDiam,InnCoil_LowEnd + (InnCoil_Height/2))
femm.mi_selectlabel(  InnCoil_InRadius + InnCoil_WireDiam,InnCoil_LowEnd + (InnCoil_Height/2))
femm.mi_setblockprop(InnCoil_WireType, 1, 0, InnCoil_Circuit, 0, 1 ,InnCoil_NrWindings)
femm.mi_clearselected()